# Prepare the data 

In [1]:
import sys,os
import numpy as np
import random
import pandas as pd

In [2]:
file = 'D:\Evans microbial community\data_for_pipeline\metadata_full_MMPRNT_G5.LC_LUX_trunc_rar_016017_v2.txt'
#label = 'D:\Evans microbial community\data_for_pipeline\ELSA_module_016017_LUX_OTU_sum_MMPRNT_G5_LC_LUX_016017.txt'
label = 'D:\Evans microbial community\data_for_pipeline\Rarefied_diversity_MMPRNT_G5_LC_LUX_016017.txt'
site = 'LUX'
site_not_used = 'LC'
cv_num = 10
test_size = 0.1 ### what the proportion of your data you want to hold out as test set, 
                ### which will never be seen when you train the model
feature_2_onehotencoding = ['FertStatus','thermal_two_year','thermal_2019','thermal_2018']
save_model_name = 'Multioutput_model_network_modules.sav'
save_result_name = 'Result_Multioutput_network_modules.txt'

In [3]:
df = pd.read_csv(file, sep='\t', index_col = 0, header = 0)
if 'ELSA' in label:
    df_lable = pd.read_csv(label, sep='\t', index_col = 42, header = 0)
if 'Rarefied' in label:
    df_lable = pd.read_csv(label, sep='\t', index_col = 0, header = 0)
ML_matrix = pd.concat([df_lable,df], axis=1, sort = False)

In [4]:
df_target_site = ML_matrix.loc[ML_matrix['siteID']==site]
df_target_site = df_target_site.drop(["collectionDate","siteID",\
                                             "UTM_Lat_Cord","UTM_Lon_Cord"],axis=1)

df_other_site = ML_matrix.loc[ML_matrix['siteID']==site_not_used]
df_other_site = df_other_site.drop(["collectionDate","siteID",\
                                             "UTM_Lat_Cord","UTM_Lon_Cord"],axis=1)

df_target_site["thermal_two_year"].isna().sum()

22

# Split the data to traning and testing

In [5]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df_target_site, \
                                       test_size=test_size, random_state=42)
X_train = train_set.drop(df_lable.columns, axis=1) 
X_test = test_set.drop(df_lable.columns, axis=1)
X_on_test_site = df_other_site.drop(df_lable.columns, axis=1)

y_train = train_set[df_lable.columns]
y_test = test_set[df_lable.columns]
y_on_test_site = df_other_site[df_lable.columns]

# OneHotEncoding, handle with NaN data (keep them as NaN)


#### Fit on training

In [6]:
from sklearn.preprocessing import OneHotEncoder
def OneHotEncoder_fit_transform(df,feature_2_onehotencoding):
    new_columns = []
    Onehot = {}
    col_2_1hot = df.loc[:,feature_2_onehotencoding]
    for col in feature_2_onehotencoding:
        Onehot[col] = OneHotEncoder(sparse=False, handle_unknown='ignore')
        c_1hot_use = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].notna(),col])
        c_1hot_na = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].isna(),col])
        if c_1hot_na.shape[0] == 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].fit_transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = col_2_1hot.index
            for columns in c_1hot_encoded.columns:
                new_columns.append(columns)
        if c_1hot_na.shape[0] != 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].fit_transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = c_1hot_use.index
            for columns in c_1hot_encoded.columns:
                c_1hot_na[columns] = np.nan
                new_columns.append(columns)
            c_1hot_na = c_1hot_na.drop(col,axis=1)
            c_1hot_encoded = pd.concat([c_1hot_encoded,c_1hot_na],axis=0)
        df = pd.concat([df,c_1hot_encoded],axis=1)
    return(df,Onehot,new_columns)

#### transform on test

In [7]:
def OneHotEncoder_transform(df,feature_2_onehotencoding,Onehot):
    col_2_1hot = df.loc[:,feature_2_onehotencoding]
    for col in feature_2_onehotencoding:
        c_1hot_use = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].notna(),col])
        c_1hot_na = pd.DataFrame(col_2_1hot.loc[col_2_1hot.loc[:,col].isna(),col])
        if c_1hot_na.shape[0] == 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = col_2_1hot.index
            df = pd.concat([df,c_1hot_encoded],axis=1)
        if c_1hot_na.shape[0] != 0 and c_1hot_use.shape[0] != 0:
            c_1hot_encoded = pd.DataFrame(Onehot[col].transform(c_1hot_use))
            c_1hot_encoded.columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            c_1hot_encoded.index = c_1hot_use.index
            for columns in c_1hot_encoded.columns:
                c_1hot_na[columns] = np.nan
            c_1hot_na = c_1hot_na.drop(col,axis=1)
            c_1hot_encoded = pd.concat([c_1hot_encoded,c_1hot_na],axis=0)
            df = pd.concat([df,c_1hot_encoded],axis=1)
        if c_1hot_use.shape[0] == 0:
            columns = [col + '_' + '%s'%sub for sub in Onehot[col].categories_[0]]
            for column in columns:
                c_1hot_na[column] = np.nan
            c_1hot_na = c_1hot_na.drop(col,axis=1)  
            df = pd.concat([df,c_1hot_na],axis=1)
    return(df)    

In [8]:
X_train, Onehot, new_columns = OneHotEncoder_fit_transform(X_train,feature_2_onehotencoding)
X_test = OneHotEncoder_transform(X_test,feature_2_onehotencoding,Onehot)
X_on_test_site = OneHotEncoder_transform(X_on_test_site,feature_2_onehotencoding,Onehot)

D:\softinstall\Anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\softinstall\Anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\softinstall\Anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 

### drop the original columns

In [9]:
X_train = X_train.drop(feature_2_onehotencoding,axis=1)
X_test = X_test.drop(feature_2_onehotencoding,axis=1)
X_on_test_site = X_on_test_site.drop(feature_2_onehotencoding,axis=1)

# Impute missing data using KNN, five Ks

### 1. drop features with >50% missing data

In [10]:
Miss_count = X_train.count(0)
Col_to_drop = Miss_count[Miss_count <= 0.5*X_train.shape[0]].index.tolist()
Col_to_drop
X_train.drop(Col_to_drop,axis=1,inplace=True)
X_test.drop(Col_to_drop,axis=1,inplace=True)
X_on_test_site.drop(Col_to_drop,axis=1,inplace=True)

### 2. impute

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
class KNNImputer_Ks(BaseEstimator, TransformerMixin):
    def __init__(self, *Ks):
        self.Ks = Ks
    def fit(self, X,Ks):
        D_imputer = {}        
        for k in [3,4,5,6,7]:
            imputer = KNNImputer(n_neighbors=k)
            D_imputer[k] = imputer.fit(X)              
        return D_imputer
    def transform(self, X):
        Impute_train = {}
        for k in [3,4,5,6,7]:
            Impute_train[k] = pd.DataFrame(D_imputer[k].transform(X))
            Impute_train[k].index = X.index
            Impute_train[k].columns = X.columns 
            if k == 3:
                Imputed = Impute_train[k].copy(deep=True)
                Imputed.loc[:,:] = 0
            Imputed = Imputed.add(Impute_train[k],fill_value=0)
        return Imputed/5

In [12]:
imputer_knn = KNNImputer_Ks()
D_imputer = imputer_knn.fit(X_train, Ks="3,4,5,6,7")
X_train_KNN = imputer_knn.transform(X_train)
X_test_KNN = imputer_knn.transform(X_test)
X_on_test_site_KNN =  imputer_knn.transform(X_on_test_site)

### round the imputed values for binary features

In [13]:
for col in new_columns:
    X_train_KNN[col] = round(X_train_KNN[col],0)
    X_test_KNN[col] = round(X_test_KNN[col],0)
    X_on_test_site_KNN[col] = round(X_on_test_site_KNN[col],0)

# Machine learning

### 1. GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score
if y_train.shape[1] == 1:
    param_grid = {'max_depth':[3, 5, 10], \
              'max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
              'n_estimators': [10, 100,500,1000]}
    Reg_Mol = RandomForestRegressor()
    grid_search = GridSearchCV(Reg_Mol, param_grid, cv=cv_num, \
                           scoring='neg_mean_squared_error', verbose=2,n_jobs=5)
    grid_search.fit(X_train_KNN, y_train)
if y_train.shape[1] > 1:    
    gsc = GridSearchCV(
                estimator=RandomForestRegressor(),
                #param_grid={'max_depth':[3, 5, 10], \
                #  'max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
                #  'n_estimators': [10, 100,500,1000]},
                param_grid={'max_depth':[3, 5, 10], \
                  'max_features': [0.1], \
                  'n_estimators': [10]},        
                cv=cv_num, scoring='neg_mean_squared_error', verbose=2, n_jobs=5)
    grid_search = MultiOutputRegressor(gsc).fit(X_train_KNN, y_train)

In [ ]:
grid_search.estimators_[2].best_estimator_
grid_search.estimators_[2].best_score_

In [46]:
def PCC_multioutput(a,b):
    R = []
    for i in range(0,a.shape[1]):
        R.append(np.corrcoef(np.array(a.loc[:,i]), np.array(b.loc[:,i]))[0,1])
                 
    return (np.array(R))

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
cv_pred = pd.DataFrame(cross_val_predict(estimator=grid_search, X=X_train_KNN, y=y_train, cv=cv_num))
cv_pred.columns = y_train.columns
cv_pred.index = y_train.index

In [44]:
cv_mse = mean_squared_error(y_train, cv_pred,multioutput='raw_values')
cv_evs = explained_variance_score(y_train, cv_pred,multioutput='raw_values')
cv_r2 = r2_score(y_train, cv_pred,multioutput='raw_values')

In [ ]:
a = y_train
b 

In [47]:
cv_cor = PCC_multioutput(y_train, cv_pred)

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [0] of <class 'int'>

In [38]:
y_train

,full_Richness,full_InvSimpson,full_Evenness
AH11,2326,297.720060,0.127997
MMPRNT235,2417,276.131726,0.114246
MMPRNT6,2095,196.192300,0.093648
MMPRNT157,2549,267.525602,0.104953
MMPRNT243,2178,47.615692,0.021862
...,...,...,...
MMPRNT245,2316,226.449275,0.097776
MMPRNT302,1872,162.028336,0.086554
AF18,2066,142.413626,0.068932
F18,2464,246.901388,0.100203


In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score
if y_train.shape[1] == 1:
    param_grid = {'max_depth':[3, 5, 10], \
              'max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
              'n_estimators': [10, 100,500,1000]}
    Reg_Mol = RandomForestRegressor()
    
if y_train.shape[1] > 1:
    param_grid = {'estimator__max_depth':[3], \
              'estimator__max_features': [None], \
              'estimator__n_estimators': [10, 100,500,1000]}    
    Reg_Mol = MultiOutputRegressor(RandomForestRegressor(), n_jobs=5)

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score
if y_train.shape[1] == 1:
    param_grid = {'max_depth':[3, 5, 10], \
              'max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
              'n_estimators': [10, 100,500,1000]}
    Reg_Mol = RandomForestRegressor()
    
if y_train.shape[1] > 1:
    param_grid = {'estimator__max_depth':[3, 5, 10], \
              'estimator__max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
              'estimator__n_estimators': [10, 100,500,1000]}    
    Reg_Mol = MultiOutputRegressor(RandomForestRegressor(), n_jobs=5)

In [57]:
grid_result.estimators_[2].best_params_

{'max_depth': 3, 'max_features': None, 'n_estimators': 500}

In [ ]:
grid_search = GridSearchCV(Reg_Mol, param_grid, cv=cv_num, \
                           scoring='neg_mean_squared_error', verbose=2,n_jobs=5)
grid_search.fit(X_train_KNN, y_train)

In [58]:
grid_search = grid_result

### 2. cross-validation and prediction

In [353]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
parameter2use = grid_search.best_params_

if y_train.shape[1] == 1:
    Reg = RandomForestRegressor(n_estimators=parameter2use['n_estimators'],\
                            max_depth=parameter2use['max_depth'],\
                            max_features= parameter2use['max_features'],\
                            criterion='mse', random_state=42)    
if y_train.shape[1] > 1:
    Reg = MultiOutputRegressor(RandomForestRegressor(n_estimators=parameter2use['estimator__n_estimators'],\
                            max_depth=parameter2use['estimator__max_depth'],\
                            max_features= parameter2use['estimator__max_features'],\
                            criterion='mse', random_state=42), n_jobs=5)

In [378]:
# mse: Mean squared error regression loss
# evs: Explained variance regression score
# r2: (coefficient of determination) regression score. 
    # Best possible score is 1.0 and it can be negative 
    # (because the model can be arbitrarily worse). 
    # A constant model that always predicts the expected value of y, 
    # disregarding the input features, would get a R^2 score of 0.0.
# cor: Pearson Correlation Coefficient between true y and predicted y
if y_train.shape[1] == 1:
    cv_pred = cross_val_predict(estimator=Reg, X=X_train_KNN, y=y_train, cv=cv_num)
    cv_mse = mean_squared_error(y_train, cv_pred)
    cv_evs = explained_variance_score(y_train, cv_pred)
    cv_r2 = r2_score(y_train, cv_pred)
    cv_cor = np.corrcoef(np.array(y_train), cv_pred)[0,1]
if y_train.shape[1] > 1:
    cv_pred = pd.DataFrame(cross_val_predict(estimator=Reg, X=X_train_KNN, y=y_train, cv=cv_num))
    cv_pred.columns = y_train.columns
    cv_pred.index = y_train.index
    cv_mse = mean_squared_error(y_train, cv_pred,multioutput='raw_values')
    cv_evs = explained_variance_score(y_train, cv_pred,multioutput='raw_values')
    cv_r2 = r2_score(y_train, cv_pred,multioutput='raw_values')
    cv_cor = np.corrcoef(np.array(y_train), cv_pred)[0,1]
    
# need to figure out how to get the multioutput PCC

In [381]:
cv_evs

array([-0.01499697, -0.02130138, -0.02601508, -0.02418396, -0.01632604,
       -0.00672137, -0.01345846, -0.01587304, -0.02855328,  0.00227371,
       -0.01996841, -0.02209687, -0.00190153, -0.02549373, -0.01205259,
       -0.02363729, -0.02879485, -0.0216127 , -0.01878749, -0.01627443,
       -0.01860381, -0.01431295, -0.01635258, -0.00749156, -0.01046488,
       -0.018761  , -0.02371008, -0.01564974, -0.00690794, -0.01592981,
        0.00831275, -0.0267923 , -0.01313112, -0.02191073, -0.0210646 ,
       -0.01413702,  0.0021471 , -0.0128265 , -0.00231004, -0.03602125,
       -0.02395407, -0.01610284])

### 3.Predict

In [382]:
Reg.fit(X_train_KNN,y_train)
pred_train = Reg.predict(X_train_KNN)
if y_train.shape[1] == 1:
    train_mse = mean_squared_error(y_train, pred_train)
    train_evs = explained_variance_score(y_train, pred_train)
    train_r2 = r2_score(y_train, pred_train)
    train_cor = np.corrcoef(np.array(y_train), pred_train)[0,1]
if y_train.shape[1] > 1:
    pred_train = pd.DataFrame(pred_train)
    pred_train.columns = y_train.columns
    pred_train.index = y_train.index
    train_mse = mean_squared_error(y_train, pred_train,multioutput='raw_values')
    train_evs = explained_variance_score(y_train, pred_train,multioutput='raw_values')
    train_r2 = r2_score(y_train, pred_train,multioutput='raw_values')
    train_cor = np.corrcoef(np.array(y_train), pred_train)[0,1]    

In [383]:
pred_test = Reg.predict(X_test_KNN)
if y_train.shape[1] == 1:
    test_mse = mean_squared_error(y_test, pred_test)
    test_evs = explained_variance_score(y_test, pred_test)
    test_r2 = r2_score(y_test, pred_test)
    test_cor = np.corrcoef(np.array(y_test), pred_test)
if y_train.shape[1] > 1:
    pred_test = pd.DataFrame(pred_test)
    pred_test.columns = y_test.columns
    pred_test.index = y_test.index
    test_mse = mean_squared_error(y_test, pred_test,multioutput='raw_values')
    test_evs = explained_variance_score(y_test, pred_test,multioutput='raw_values')
    test_r2 = r2_score(y_test, pred_test,multioutput='raw_values')
    test_cor = np.corrcoef(np.array(y_test), pred_test)[0,1]        

In [385]:
pred_on_test_site = Reg.predict(X_on_test_site_KNN)
if y_train.shape[1] == 1:
    test_site_mse = mean_squared_error(y_on_test_site, pred_on_test_site)
    test_site_evs = explained_variance_score(y_on_test_site, pred_on_test_site)
    test_site_r2 = r2_score(y_on_test_site, pred_on_test_site)
    test_site_cor = np.corrcoef(np.array(y_on_test_site), pred_on_test_site)[0,1]
if y_train.shape[1] > 1:
    pred_on_test_site = pd.DataFrame(pred_on_test_site)
    pred_on_test_site.columns = y_on_test_site.columns
    pred_on_test_site.index = y_on_test_site.index
    test_site_mse = mean_squared_error(y_on_test_site, pred_on_test_site,multioutput='raw_values')
    test_site_evs = explained_variance_score(y_on_test_site, pred_on_test_site,multioutput='raw_values')
    test_site_r2 = r2_score(y_on_test_site, pred_on_test_site,multioutput='raw_values')
    test_site_cor = np.corrcoef(np.array(y_on_test_site), pred_on_test_site)[0,1]            

# Parse the results

### Save the fitted model

In [369]:
import pickle
pickle.dump(Reg, open(save_model_name, 'wb'))

### write the hyperparameters

In [ ]:
out = open(save_result_name,'w')
out.write('The model is built using data from %s, and applied to %s and %s.\n\n'%(site,site,site_not_used))
out.write('There are %s training instances.\n'%X_train_KNN.shape[0])
out.write('There are %s test instances.\n'%X_test_KNN.shape[0])
out.write('There are %s instances in the other site.\n\n'%X_on_test_site_KNN.shape[0])
out.write('The model is built using RandomForestRegressor, with:\n')
if y_train.shape[1] == 1:
    out.write('\tn_estimators: %s\n'%parameter2use['n_estimators'])
    out.write('\tmax_depth: %s\n'%parameter2use['max_depth'])
    out.write('\tmax_features: %s\n\n'%parameter2use['max_features'])
if y_train.shape[1] > 1:
    out.write('\tn_estimators: %s\n'%parameter2use['estimator__n_estimators'])
    out.write('\tmax_depth: %s\n'%parameter2use['estimator__max_depth'])
    out.write('\tmax_features: %s\n\n'%parameter2use['estimator__max_features'])    
out.write('There are %s feature used\n\n'%X_train_KNN.shape[1])

### Performance of models

In [ ]:
out.write('Prediction\tmse\tevs\tr2\tPCC\n')
out.write('CV\t%s\t%s\t%s\t%s\n'%(cv_mse,cv_evs,cv_r2,cv_cor))
out.write('Train\t%s\t%s\t%s\t%s\n'%(train_mse,train_evs,train_r2,train_cor))
out.write('Test\t%s\t%s\t%s\t%s\n'%(test_mse,test_evs,test_r2,test_cor))
out.write('Other_site\t%s\t%s\t%s\t%s\n\n'%(test_site_mse,test_site_evs,test_site_r2,test_site_cor))
out.close()

### Important features

In [ ]:
imp = pd.DataFrame({'Feature':X_train_KNN.columns, 'Importance':Reg.feature_importances_})
imp_sorted = imp.sort_values(by='Importance', ascending=False)
imp_sorted.to_csv(save_result_name.split('.txt')[0] + '_imp.txt', index=False, header=True,sep="\t")

In [ ]:
# Add the importance direction in the output